# keras概述

Keras是基于Theano和TensorFlow的深度学习库，其特点如下：

1、简易和快速的原型设计（keras具有高度模块化，极简，和可扩充特性）  
2、支持CNN和RNN，或二者的结合  
3、支持任意的链接方案（包括多输入和多输出训练）  
4、无缝CPU和GPU切换

Keras的核心数据结构是“模型”，模型是一种组织网络层的方式，Keras中有两种模型：  
Sequential模型：序贯模型，也就是单输入单输出，一条路通到底，层与层之间只有相邻关系，跨层连接统统没有。    
functional模型：函数模型

第一步、选择模型 
from keras.models import    
Sequential model = Sequential()  

第二步、搭建模型：将一些网络层通过.add()堆叠起来，就构成了一个模型：  
from keras.layers import Dense, Activation  
model.add(Dense(output_dim=64, input_dim=100))  
model.add(Activation("relu"))  
model.add(Dense(output_dim=10))  
model.add(Activation("softmax"))  

第三步、编译模型：compile()，需指明损失函数、优化器、评估指标，可自己定制优化器  
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])  

第四步、训练模型： fit（）操作 或 train_on_batch（）操作  
fit（）操作：在训练数据上按batch进行一定次数的迭代训练，以拟合网络。  
model.fit(X_train, Y_train, nb_epoch=5, batch_size=32)  
train_on_batch（）操作：手动将一个个batch的数据送入网络中训练  
model.train_on_batch(X_batch, Y_batch)  

第五步、评估模型： evaluate返回损失值和compilie定义的评估指标、predict、predict_classes预测输出，predict_proba返回预测值的概率。  
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)  
classes = model.predict_classes(X_test, batch_size=32)  
classes = model.predict(X_test)  
proba = model.predict_proba(X_test, batch_size=32)

# 模型的保存

In [ ]:
# 保存模型的结构、权重、配置
from keras.models import load_model
 
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model
 
# returns a compiled model
# identical to the previous one
model = load_model('my_model.h5')

In [ ]:
# 只保存模型的结构
from keras.models import model_from_json

# save as JSON
json_string = model.to_json()
# save as YAML
yaml_string = model.to_yaml()

# model reconstruction from JSON:
model = model_from_json(json_string)
# model reconstruction from YAML
model = model_from_yaml(yaml_string)

In [ ]:
#保存模型的权重

model.save_weights('my_model_weights.h5')

#在代码中初始化一个完全相同的模型
model.load_weights('my_model_weights.h5')
#加载权重到不同的网络结构（有些层一样）
model.load_weights('my_model_weights.h5', by_name=True)

"""
假如原模型为：
    model = Sequential()
    model.add(Dense(2, input_dim=3, name="dense_1"))
    model.add(Dense(3, name="dense_2"))
    ...
    model.save_weights(fname)
"""
# new model
model = Sequential()
model.add(Dense(2, input_dim=3, name="dense_1"))  # will be loaded
model.add(Dense(10, name="new_dense"))  # will not be loaded
 
# load weights from first model; will only affect the first layer, dense_1.
model.load_weights(fname, by_name=True)

# one-hot编码

In [22]:
from keras.utils.np_utils import to_categorical
from numpy import argmax

#数据为整型或整型字符的列表（数组）时，直接通过to_categorical编码
int_labels=[0,2,1,3]
# int_labels=['0','2','1','3']
categorical_labels = to_categorical(int_labels, num_classes=4) #编码
categorical_labels
inverted = argmax(categorical_labels[0]) #反编码
inverted

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]], dtype=float32)

0

In [24]:
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from numpy import argmax

#数据为其他字符串类型的列表（数组）时，先通过LabelEncoder标准化标签，再通过to_categorical编码
str_labels=['a','b','c','a']

encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(str_labels) #标准化标签，将标签值统一转换成range(标签值个数-1)范围内
encoded_labels
categorical_labels = to_categorical(encoded_labels, num_classes=3) #编码
categorical_labels
inverted = argmax(categorical_labels[0]) #反编码
inverted

array([0, 1, 2, 0], dtype=int64)

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]], dtype=float32)

0

# 案列一：神经网络解决回归问题

In [ ]:
import numpy as np
np.random.seed(1337)
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt

# 1.Build the trainning data
X=np.linspace(-1,1,200)
np.random.shuffle(X)
Y=0.5*X+2+np.random.normal(0,0.05,(200,))
plt.scatter(X,Y)
plt.show()
X_train,Y_train=X[:160],Y[:160]
X_test,Y_test=X[160:],Y[160:]
print(X)
print("*******************************************")
print(Y)

# 2.Build a neural network from the 1st layer to the last layer
model=Sequential()
model.add(Dense(output_dim=1,input_dim=1))

#3. Choose loss function and optimzing method
model.compile(loss='mse',optimizer='sgd')

#4. Trainning
print("Training......")
for step in range(1400):
        cost=model.train_on_batch(X_train,Y_train)
        if step % 100 ==0:
                print('train cost',cost)
                
#5.Test
print("\n Testing...........")
cost=model.evaluate(X_test,Y_test,batch_size=40)
print("Test cost:",cost)
W,b=model.layers[0].get_weights()
print('Weight=',W,"\nbiases=",b)

#6.Plotting the prediction
Y_pred=model.predict(X_test)
plt.scatter(X_test,Y_test)
plt.plot(X_test,Y_pred)
plt.show()

# 案列二：神经网络解决二类分类问题

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
#输入训练数据 keras接收numpy数组类型的数据
x=np.array([[0,1,0],
            [0,0,1],
            [1,3,2],
            [3,2,1]])
y=np.array([0,0,1,1]).T
#最简单的序贯模型，序贯模型是多个网络层的线性堆叠
simple_model=Sequential()
#dense层为全连接层
#第一层隐含层为全连接层 5个神经元 输入数据的维度为3
simple_model.add(Dense(5,input_dim=3,activation='relu'))
#第二个隐含层 4个神经元
simple_model.add(Dense(4,activation='relu'))
#输出层为1个神经元
simple_model.add(Dense(1,activation='sigmoid'))
#编译模型,训练模型之前需要编译模型
#编译模型的三个参数：优化器、损失函数、指标列表
simple_model.compile(optimizer='sgd',loss='mean_squared_error')
#训练网络 2000次
#Keras以Numpy数组作为输入数据和标签的数据类型。训练模型一般使用fit函数
simple_model.fit(x,y,epochs=40,batch_size=128)
#应用模型 进行预测
y_=simple_model.predict_classes(x[0:1])
print("[0,1,0]的分类结果："+str(y[0]))

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Generate dummy data
x_train = np.random.random((1000, 20))
y_train = np.random.randint(2, size=(1000, 1))
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()
model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)
print(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                1344      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1000/1000 [==============================] - 0s 106us/step - loss: 0.7362 - acc: 0.4990
Epoch 2/20
1000/1000 [===================

100/100 [==============================] - 0s 270us/step
[0.68873894214630127, 0.60000002384185791]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers.core import Dense, Activation,Dropout
from keras.utils import np_utils

#设置随机数生成器种子,实现结果的重现
seed = 42
np.random.seed(seed)

dataframe = pandas.read_csv("data/sonar.csv", header=None) # 加载数据集
dataset = dataframe.values
X = dataset[:, 0:60].astype(float) # 分割为60个输入变量
Y = dataset[:, 60]

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
train_X, test_X, train_y, test_y = train_test_split(X, encoded_Y, test_size=0.4, random_state=0)


model = Sequential()

model.add(Dense(30,input_dim=60, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(30, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.fit(train_X, train_y, epochs=250, batch_size=7, verbose=1)

loss, accuracy = model.evaluate(test_X,test_y,verbose=1)
print ("Accuracy = {:.2f}".format(accuracy))

# 案列三：神经网络解决多分类问题

In [22]:
##2.1引用包
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import metrics
import keras
##2.2生成数据
# Generate dummy data
import numpy as np
x_train = np.random.random((1000, 20))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
##2.3构建模型
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=20))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
##2.4训练模型
model.fit(x_train, y_train,
          epochs=40,
          batch_size=128)
##2.5评估模型
score=model.evaluate(x_test, y_test, batch_size=128)
print(score)

Epoch 1/40
1000/1000 [==============================] - 3s 3ms/step - loss: 2.4140 - acc: 0.1120
Epoch 2/40
1000/1000 [==============================] - 0s 42us/step - loss: 2.4050 - acc: 0.1050
Epoch 3/40
1000/1000 [==============================] - 0s 44us/step - loss: 2.3378 - acc: 0.1090
Epoch 4/40
1000/1000 [==============================] - 0s 47us/step - loss: 2.3336 - acc: 0.1050
Epoch 5/40
1000/1000 [==============================] - 0s 38us/step - loss: 2.3410 - acc: 0.1030
Epoch 6/40
1000/1000 [==============================] - 0s 49us/step - loss: 2.3241 - acc: 0.1090
Epoch 7/40
1000/1000 [==============================] - 0s 47us/step - loss: 2.3143 - acc: 0.1080
Epoch 8/40
1000/1000 [==============================] - 0s 47us/step - loss: 2.3101 - acc: 0.1180
Epoch 9/40
1000/1000 [==============================] - 0s 48us/step - loss: 2.3042 - acc: 0.1130
Epoch 10/40
1000/1000 [==============================] - 0s 50us/step - loss: 2.3101 - acc: 0.1090
Epoch 11/40
1000/100

100/100 [==============================] - 1s 14ms/step
[2.2973096370697021, 0.12999999523162842]


In [ ]:
## coding=utf-8

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils

def one_hot_encode_object_array(arr):
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))


iris = sns.load_dataset("iris")
X = iris.values[:, :4]
y = iris.values[:, 4]

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.6, random_state=0)
train_y_ohe = one_hot_encode_object_array(train_y)
test_y_ohe = one_hot_encode_object_array(test_y)


model = Sequential()

model.add(Dense(200,input_dim=4,activation='relu'))
model.add(Dense(3,activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
model.fit(train_X, train_y_ohe, epochs=20, batch_size=5, verbose=0)

# predict_y = model.predict_proba(test_X)
# print predict_y

loss, accuracy = model.evaluate(test_X,test_y_ohe,verbose=1)
print ("Accuracy = {:.2f}".format(accuracy))

# KerasClassifier方法

In [32]:
#https://www.cnblogs.com/arkenstone/p/5943489.html?utm_source=itdadao&utm_medium=referral 
#利用Keras进行多类分类
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

# load dataset
iris = sns.load_dataset("iris")
X = iris.values[:, :4]
Y = iris.values[:, 4]

# encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(Y) #标准化标签，将标签值统一转换成range(标签值个数-1)范围内
# convert integers to dummy variables (one hot encoding)
dummy_y = np_utils.to_categorical(encoded_Y)

# define model structure
def baseline_model():
    model = Sequential()
    model.add(Dense(units=200, input_dim=4, activation='relu'))
    model.add(Dense(units=3,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
estimator = KerasClassifier(build_fn=baseline_model, nb_epoch=20, batch_size=5)
# splitting data into training set and test set. If random_state is set to an integer, the split datasets are fixed.
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.3, random_state=0)
estimator.fit(X_train, Y_train)

# make predictions
pred = estimator.predict(X_test)

# inverse numeric variables to initial categorical labels
init_lables = encoder.inverse_transform(pred)

# k-fold cross-validate
seed = 42
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
results.mean()*100

Epoch 1/1
105/105 [==============================] - 4s 37ms/step - loss: 1.0683 - acc: 0.3048


Epoch 1/1
15/15 [==============================] - 1s 88ms/step
Epoch 1/1
15/15 [==============================] - 1s 90ms/step
Epoch 1/1
15/15 [==============================] - 1s 90ms/step
Epoch 1/1
15/15 [==============================] - 1s 93ms/step
Epoch 1/1
15/15 [==============================] - 1s 92ms/step
Epoch 1/1
15/15 [==============================] - 1s 91ms/step
Epoch 1/1
15/15 [==============================] - 1s 90ms/step
Epoch 1/1
15/15 [==============================] - 2s 114ms/step
Epoch 1/1
15/15 [==============================] - 1s 91ms/step
Epoch 1/1
15/15 [==============================] - 2s 101ms/step


64.666667133569717